<a href="https://colab.research.google.com/github/Stephenthomas10/Projects-and-implementations/blob/main/FPLoptimized.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df=pd.read_csv('/content/players.csv')
df.columns

Index(['id', 'name', 'now_cost', 'position', 'team', 'goals_conceded_per_90',
       'ict_index_rank_type', 'starts_per_90', 'event_points',
       'goals_conceded', 'penalties_missed', 'expected_goals_conceded_per_90',
       'chance_of_playing_this_round', 'cost_change_event_fall',
       'expected_assists_per_90', 'form_rank', 'transfers_in_event', 'starts',
       'points_per_game_rank', 'transfers_out',
       'corners_and_indirect_freekicks_order', 'bonus', 'saves_per_90',
       'points_per_game_rank_type', 'expected_goals_per_90', 'form',
       'selected_rank', 'points_per_game', 'total_points', 'in_dreamteam',
       'cost_change_start', 'selected_by_percent', 'saves',
       'transfers_out_event', 'creativity_rank_type', 'red_cards',
       'ict_index_rank', 'assists', 'goals_scored', 'creativity',
       'value_season', 'news', 'influence_rank',
       'chance_of_playing_next_round', 'minutes', 'own_goals', 'now_cost_rank',
       'cost_change_event', 'expected_assists', 'n

In [ ]:
!pip install pulp
import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable

# Load player dataset
# Replace 'players.csv' with the actual dataset path or DataFrame
df = pd.read_csv("players.csv")

# Constraints
budget = 1000  # Budget in 0.1M units (e.g., 100M = 1000 in the FPL system)
max_players_per_team = 3
formation_constraints = {"GK": 2, "DEF": 5, "MID": 5, "FWD": 3}
starting_formation = {"GK": 1, "DEF": 3, "MID": 4, "FWD": 3}
max_squad_size = 15

# Calculate expected points per player
# Replace with your model/formula for expected points
df['expected_points'] = (
    df['form'] * 0.5 +
    df['points_per_game'] * 0.3 +
    (df['expected_goals'] + df['expected_assists']) * 0.2
)

# Define the optimization problem
problem = LpProblem("FPL_Squad_Optimization", LpMaximize)

# Create decision variables for player inclusion
players = {i: LpVariable(f"player_{i}", cat="Binary") for i in df.index}

# Objective Function: Maximize total expected points
problem += sum(df.loc[i, "expected_points"] * players[i] for i in df.index), "Total_Expected_Points"

# Constraint 1: Budget limit
problem += sum(df.loc[i, "now_cost"] * players[i] for i in df.index) <= budget, "Budget_Limit"

# Constraint 2: Max squad size
problem += sum(players[i] for i in df.index) == max_squad_size, "Squad_Size_Limit"

# Constraint 3: Formation constraints
for position, max_count in formation_constraints.items():
    problem += sum(players[i] for i in df[df["position"] == position].index) <= max_count, f"Max_{position}"

# Constraint 4: Team limit
for team in df["team"].unique():
    problem += sum(players[i] for i in df[df["team"] == team].index) <= max_players_per_team, f"Max_Team_{team}"

# Solve the optimization problem
problem.solve()

# Extract the optimal solution
selected_players = [i for i in df.index if players[i].value() == 1]
optimal_squad = df.loc[selected_players]

# Separate starting 11 and bench based on expected points
optimal_squad = optimal_squad.sort_values(by="expected_points", ascending=False)
starting_11 = optimal_squad.iloc[:11]
bench = optimal_squad.iloc[11:]

# Display the results
print("Optimal Squad:")
print(optimal_squad)

print("\nStarting 11:")
print(starting_11)

print("\nBench:")
print(bench)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 66.6 MB/s eta 0:00:00
Optimal Squad:
      id                              name  now_cost position         team  \
401  328                     Mohamed Salah       132      MID    Liverpool   
13    17                       Bukayo Saka       103      MID      Arsenal   
659  541  Matheus Santos Carneiro Da Cunha        71      FWD       Wolves   
448  366            Bruno Borges Fernandes        85      MID      Man Utd   
83    71                   Justin Kluivert        53      MID  Bournemouth   
152  129     João Pedro Junqueira de Jesus        58      FWD     Brighton   
127  110                       Yoane Wissa        61      FWD    Brentford   
596  491                   Brennan Johnson        67      MID        Spurs   
621  388                 Aaron Wan-Bissaka        45      DEF     West Ham   
398  325                 Caoimhin Kelleher        45      GKP    Liverpool   
2      3      Gabriel dos Santos Magalhães      

In [ ]:
!pip install requests
import requests
import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable

# Replace with your FPL team ID
TEAM_ID = 'YOUR_TEAM_ID'

# Fetch current squad data
def get_current_squad(team_id):
    url = f'https://fantasy.premierleague.com/api/my-team/{776381}/'
    response = requests.get(url)
    data = response.json()
    return data['picks']

# Fetch player data
def get_player_data():
    url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
    response = requests.get(url)
    data = response.json()
    players = pd.DataFrame(data['elements'])
    teams = pd.DataFrame(data['teams'])
    return players, teams

# Fetch fixtures
def get_fixtures():
    url = 'https://fantasy.premierleague.com/api/fixtures/'
    response = requests.get(url)
    fixtures = pd.DataFrame(response.json())
    return fixtures

# Calculate expected points (example formula)
def calculate_expected_points(players):
    players['expected_points'] = (
        players['form'].astype(float) * 0.5 +
        players['points_per_game'].astype(float) * 0.3 +
        (players['expected_goals'].astype(float) + players['expected_assists'].astype(float)) * 0.2
    )
    return players

# Optimization function
def optimize_squad(current_squad, players, budget):
    problem = LpProblem("FPL_Squad_Optimization", LpMaximize)
    player_vars = {i: LpVariable(f"player_{i}", cat="Binary") for i in players.index}
    problem += sum(players.loc[i, 'expected_points'] * player_vars[i] for i in players.index)
    problem += sum(players.loc[i, 'now_cost'] * player_vars[i] for i in players.index) <= budget
    problem += sum(player_vars[i] for i in players.index) == 15
    for position in ['GKP', 'DEF', 'MID', 'FWD']:
        max_count = {'GKP': 2, 'DEF': 5, 'MID': 5, 'FWD': 3}[position]
        problem += sum(player_vars[i] for i in players[players['element_type'] == position].index) <= max_count
    for team in players['team'].unique():
        problem += sum(player_vars[i] for i in players[players['team'] == team].index) <= 3
    problem.solve()
    selected_players = [i for i in players.index if player_vars[i].value() == 1]
    return players.loc[selected_players]

# Main function
def main():
    current_squad = get_current_squad(TEAM_ID)
    players, teams = get_player_data()
    fixtures = get_fixtures()
    players = calculate_expected_points(players)
    budget = 1000  # Example budget in 0.1M units
    optimized_squad = optimize_squad(current_squad, players, budget)
    print("Recommended Transfers:")
    print(optimized_squad[['web_name', 'team', 'expected_points']])

if __name__ == "__main__":
    main()


KeyError: 'picks'

In [ ]:
response = requests.get(f'https://fantasy.premierleague.com/api/my-team/{776381}/')
print(response.json())


{'detail': 'Authentication credentials were not provided.'}


In [ ]:
import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable

# Load the dataset of all players
players_df = pd.read_csv("players.csv")  # Ensure this file contains all player data

# Manual input of player IDs in your current squad
current_squad_ids = [15,291,630,3,366,99,491,328,541,58,401,521,177,295,270]  # Replace with your current squad's player IDs

# Filter the current squad details from the dataset
current_squad_df = players_df[players_df['id'].isin(current_squad_ids)]

# Calculate expected points based on metrics
# Adjust the formula based on your prioritization
players_df['expected_points'] = (
    players_df['form'].astype(float) * 0.5 +
    players_df['points_per_game'].astype(float) * 0.3 +
    (players_df['expected_goals_per_90'].astype(float) + players_df['expected_assists_per_90'].astype(float)) * 0.2
)

# Define optimization problem
budget = 100  # Your remaining budget in 0.1M units (e.g., 100 = £100M)
free_transfers = 1  # Number of free transfers available

def optimize_squad(current_squad_df, players_df, budget, free_transfers):
    problem = LpProblem("FPL_Squad_Optimization", LpMaximize)

    # Create decision variables for player inclusion
    players = {i: LpVariable(f"player_{i}", cat="Binary") for i in players_df.index}

    # Objective: Maximize expected points
    problem += sum(players_df.loc[i, "expected_points"] * players[i] for i in players_df.index), "Total_Expected_Points"

    # Budget constraint
    problem += sum(players_df.loc[i, "now_cost"] * players[i] for i in players_df.index) <= budget, "Budget_Limit"

    # Squad size constraint
    problem += sum(players[i] for i in players_df.index) == 15, "Squad_Size_Limit"

    # Position constraints
    position_limits = {"DEF": 5, "MID": 5, "FWD": 3, "GKP": 2}
    for position, max_count in position_limits.items():
        problem += sum(players[i] for i in players_df[players_df["position"] == position].index) <= max_count, f"Max_{position}"

    # Solve the optimization problem
    problem.solve()

    # Extract the optimal solution
    selected_players = [i for i in players_df.index if players[i].value() == 1]
    optimal_squad = players_df.loc[selected_players]
    return optimal_squad

# Run optimization
optimized_squad = optimize_squad(current_squad_df, players_df, budget, free_transfers)

# Display results
print("Recommended Squad:")
print(optimized_squad[['id', 'web_name', 'team', 'position', 'now_cost', 'expected_points']])


Recommended Squad:
      id    web_name         team position  now_cost  expected_points
16    21     Tierney      Arsenal      DEF        43            0.000
28   657     Kacurri      Arsenal      DEF        40            0.000
29   658      Heaven      Arsenal      DEF        40            0.000
31   662     Nichols      Arsenal      DEF        40            0.000
37    32        Cash  Aston Villa      DEF        44            0.796
..   ...         ...          ...      ...       ...              ...
678  561  Pedro Lima       Wolves      DEF        40            0.000
681  564     S.Bueno       Wolves      DEF        43            1.518
684  567        Toti       Wolves      DEF        43            2.306
685  637    Meupiyou       Wolves      DEF        40            0.000
688  664        Pond       Wolves      DEF        39            0.450

[151 rows x 6 columns]


In [ ]:
import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable

# Load the dataset of all players
players_df = pd.read_csv("players.csv")  # Ensure this file contains all player data

# Manual input of player IDs in your current squad
current_squad_ids = [15,291,630,3,366,99,491,328,541,58,401,521,177,295,270]  # Replace with your current squad's player IDs

# Filter the current squad details from the dataset
current_squad_df = players_df[players_df['id'].isin(current_squad_ids)]

# Filter out inactive or irrelevant players
players_df = players_df[players_df['chance_of_playing_this_round'] > 50]
players_df = players_df[players_df['form'] > 0]

# Calculate expected points based on metrics
players_df['expected_points'] = (
    players_df['form'].astype(float) * 0.5 +
    players_df['points_per_game'].astype(float) * 0.3 +
    (players_df.get('expected_goals_per_90', 0).astype(float) + players_df.get('expected_assists_per_90', 0).astype(float)) * 0.2
)

# Define optimization problem
budget = 100  # Your remaining budget in 0.1M units (e.g., 100 = £100M)
free_transfers = 1  # Number of free transfers available

def optimize_squad(players_df, budget):
    problem = LpProblem("FPL_Squad_Optimization", LpMaximize)

    # Create decision variables for player inclusion
    players = {i: LpVariable(f"player_{i}", cat="Binary") for i in players_df.index}

    # Objective: Maximize expected points
    problem += sum(players_df.loc[i, "expected_points"] * players[i] for i in players_df.index), "Total_Expected_Points"

    # Budget constraint
    problem += sum(players_df.loc[i, "now_cost"] * players[i] for i in players_df.index) <= budget, "Budget_Limit"

    # Squad size constraint
    problem += sum(players[i] for i in players_df.index) == 15, "Squad_Size_Limit"

    # Position constraints
    position_limits = {"DEF": 5, "MID": 5, "FWD": 3, "GKP": 2}
    for position, max_count in position_limits.items():
        problem += sum(players[i] for i in players_df[players_df["position"] == position].index) <= max_count, f"Max_{position}"

    # Solve the optimization problem
    problem.solve()

    # Extract the optimal solution
    selected_players = [i for i in players_df.index if players[i].value() == 1]
    optimal_squad = players_df.loc[selected_players]
    return optimal_squad

# Run optimization
optimized_squad = optimize_squad(players_df, budget)

# Transfer recommendations with position and budget constraints
def get_transfer_recommendations(current_squad_df, optimized_squad, free_transfers, budget):
    current_ids = set(current_squad_df['id'])
    optimized_ids = set(optimized_squad['id'])

    # Players to transfer out and in
    transfers_out = current_ids - optimized_ids
    transfers_in = optimized_ids - current_ids

    out_players = players_df[players_df['id'].isin(transfers_out)]
    in_players = players_df[players_df['id'].isin(transfers_in)]

    # Position-matching transfers
    position_transfers = []
    remaining_budget = budget + out_players['now_cost'].sum()

    for _, out_player in out_players.iterrows():
        position = out_player['position']
        possible_replacements = in_players[in_players['position'] == position]
        affordable_replacements = possible_replacements[possible_replacements['now_cost'] <= remaining_budget]

        if not affordable_replacements.empty:
            best_replacement = affordable_replacements.sort_values(by='expected_points', ascending=False).iloc[0]
            position_transfers.append((out_player, best_replacement))
            remaining_budget -= best_replacement['now_cost']

    # Hypothetical transfers ignoring budget
    hypothetical_transfers = [
        (out_player, in_players[in_players['position'] == out_player['position']].sort_values(by='expected_points', ascending=False).iloc[0])
        for _, out_player in out_players.iterrows()
        if not in_players[in_players['position'] == out_player['position']].empty
    ]

    return position_transfers, hypothetical_transfers

position_transfers, hypothetical_transfers = get_transfer_recommendations(
    current_squad_df, optimized_squad, free_transfers, budget
)

# Display results
print("Budget-Constrained Transfer Recommendations:")
for out_player, in_player in position_transfers:
    print(f"OUT: {out_player['web_name']} ({out_player['team']}, {out_player['position']}, {out_player['now_cost']})")
    print(f"IN: {in_player['web_name']} ({in_player['team']}, {in_player['position']}, {in_player['now_cost']})\n")

print("\nHypothetical Transfers (Ignoring Budget):")
for out_player, in_player in hypothetical_transfers:
    print(f"OUT: {out_player['web_name']} ({out_player['team']}, {out_player['position']}, {out_player['now_cost']})")
    print(f"IN: {in_player['web_name']} ({in_player['team']}, {in_player['position']}, {in_player['now_cost']})\n")


Budget-Constrained Transfer Recommendations:
OUT: Gabriel (Arsenal, DEF, 62)
IN: Tuanzebe (Ipswich, DEF, 40)

OUT: Raya (Arsenal, GKP, 55)
IN: Muric (Ipswich, GKP, 44)

OUT: Watkins (Aston Villa, FWD, 90)
IN: Daka (Leicester, FWD, 48)

OUT: Mbeumo (Brentford, MID, 78)
IN: Brooks (Bournemouth, MID, 49)

OUT: B.Fernandes (Man Utd, MID, 85)
IN: Brooks (Bournemouth, MID, 49)

OUT: Isak (Newcastle, FWD, 85)
IN: Daka (Leicester, FWD, 48)


Hypothetical Transfers (Ignoring Budget):
OUT: Gabriel (Arsenal, DEF, 62)
IN: Tuanzebe (Ipswich, DEF, 40)

OUT: Raya (Arsenal, GKP, 55)
IN: Muric (Ipswich, GKP, 44)

OUT: Watkins (Aston Villa, FWD, 90)
IN: Daka (Leicester, FWD, 48)

OUT: Mbeumo (Brentford, MID, 78)
IN: Brooks (Bournemouth, MID, 49)

OUT: B.Fernandes (Man Utd, MID, 85)
IN: Brooks (Bournemouth, MID, 49)

OUT: Isak (Newcastle, FWD, 85)
IN: Daka (Leicester, FWD, 48)



In [ ]:
import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable
import requests

# Load the dataset of all players
players_df = pd.read_csv("players.csv")  # Ensure this file contains all player data

# Manual input of player IDs in your current squad
current_squad_ids = [15, 291, 630, 3, 366, 99, 491, 328, 541, 58, 401, 521, 177, 295, 270]  # Replace with your current squad's player IDs

# Filter the current squad details from the dataset
current_squad_df = players_df[players_df['id'].isin(current_squad_ids)]

# Fetch fixture data from FPL API
url = "https://fantasy.premierleague.com/api/fixtures/"
response = requests.get(url)
fixtures = response.json()

# Convert fixtures to DataFrame
fixtures_df = pd.DataFrame(fixtures)
fixtures_df = fixtures_df[['event', 'team_h', 'team_a', 'team_h_difficulty', 'team_a_difficulty']]

# Team ID to name mapping for the 2024/25 Premier League season
team_mapping = {
    1: "Arsenal", 2: "Aston Villa", 3: "Bournemouth", 4: "Brentford", 5: "Brighton & Hove Albion",
    6: "Chelsea", 7: "Crystal Palace", 8: "Everton", 9: "Fulham", 10: "Liverpool",
    11: "Manchester City", 12: "Manchester United", 13: "Newcastle United", 14: "Nottingham Forest",
    15: "Tottenham Hotspur", 16: "West Ham United", 17: "Wolverhampton Wanderers", 18: "Leicester City",
    19: "Ipswich Town", 20: "Southampton"
}

# Map team IDs to names
fixtures_df['home_team'] = fixtures_df['team_h'].map(team_mapping)
fixtures_df['away_team'] = fixtures_df['team_a'].map(team_mapping)

# Calculate average fixture difficulty for each team
current_gameweek = 14  # Replace with current Gameweek
n = 5  # Number of upcoming Gameweeks to consider
upcoming_fixtures = fixtures_df[fixtures_df['event'] <= current_gameweek + n]

team_fdr = {}
for team in team_mapping.values():
    home_fixtures = upcoming_fixtures[upcoming_fixtures['home_team'] == team]
    away_fixtures = upcoming_fixtures[upcoming_fixtures['away_team'] == team]
    avg_difficulty = (home_fixtures['team_h_difficulty'].mean() + away_fixtures['team_a_difficulty'].mean()) / 2
    team_fdr[team] = avg_difficulty

# Filter out inactive or irrelevant players
players_df = players_df[players_df['chance_of_playing_this_round'] > 50]
players_df = players_df[players_df['form'] > 0]

# Calculate expected points with FDR adjustment
players_df['expected_points'] = (
    players_df['form'].astype(float) * 0.5 +
    players_df['points_per_game'].astype(float) * 0.3 +
    (players_df.get('expected_goals_per_90', 0).astype(float) + players_df.get('expected_assists_per_90', 0).astype(float)) * 0.2
) * (6 - players_df['team'].map(team_fdr)) / 5  # Adjust by fixture difficulty

# Handle missing or invalid values
players_df['form'] = players_df['form'].fillna(0)
players_df['points_per_game'] = players_df['points_per_game'].fillna(0)
players_df['expected_points'] = players_df['expected_points'].replace([pd.NA, None], 0).fillna(0)

# Define optimization problem
budget = 100  # Your remaining budget in 0.1M units (e.g., 100 = £100M)
free_transfers = 1  # Number of free transfers available

def optimize_squad(players_df, budget):
    problem = LpProblem("FPL_Squad_Optimization", LpMaximize)

    # Create decision variables for player inclusion
    players = {i: LpVariable(f"player_{i}", cat="Binary") for i in players_df.index}

    # Objective: Maximize expected points
    problem += sum(players_df.loc[i, "expected_points"] * players[i] for i in players_df.index), "Total_Expected_Points"

    # Budget constraint
    problem += sum(players_df.loc[i, "now_cost"] * players[i] for i in players_df.index) <= budget, "Budget_Limit"

    # Squad size constraint
    problem += sum(players[i] for i in players_df.index) == 15, "Squad_Size_Limit"

    # Position constraints
    position_limits = {"DEF": 5, "MID": 5, "FWD": 3, "GKP": 2}
    for position, max_count in position_limits.items():
        problem += sum(players[i] for i in players_df[players_df["position"] == position].index) <= max_count, f"Max_{position}"

    # Solve the optimization problem
    problem.solve()

    # Extract the optimal solution
    selected_players = [i for i in players_df.index if players[i].value() == 1]
    optimal_squad = players_df.loc[selected_players]
    return optimal_squad

# Run optimization
optimized_squad = optimize_squad(players_df, budget)

# Transfer recommendations with position and budget constraints
def get_transfer_recommendations(current_squad_df, optimized_squad, free_transfers, budget):
    current_ids = set(current_squad_df['id'])
    optimized_ids = set(optimized_squad['id'])

    # Players to transfer out and in
    transfers_out = current_ids - optimized_ids
    transfers_in = optimized_ids - current_ids

    out_players = players_df[players_df['id'].isin(transfers_out)]
    in_players = players_df[players_df['id'].isin(transfers_in)]

    # Position-matching transfers
    position_transfers = []
    remaining_budget = budget + out_players['now_cost'].sum()

    for _, out_player in out_players.iterrows():
        position = out_player['position']
        possible_replacements = in_players[in_players['position'] == position]
        affordable_replacements = possible_replacements[possible_replacements['now_cost'] <= remaining_budget]

        if not affordable_replacements.empty:
            best_replacement = affordable_replacements.sort_values(by='expected_points', ascending=False).iloc[0]
            position_transfers.append((out_player, best_replacement))
            remaining_budget -= best_replacement['now_cost']

    # Hypothetical transfers ignoring budget
    hypothetical_transfers = [
        (out_player, in_players[in_players['position'] == out_player['position']].sort_values(by='expected_points', ascending=False).iloc[0])
        for _, out_player in out_players.iterrows()
        if not in_players[in_players['position'] == out_player['position']].empty
    ]

    return position_transfers, hypothetical_transfers

position_transfers, hypothetical_transfers = get_transfer_recommendations(
    current_squad_df, optimized_squad, free_transfers, budget
)

# Display results
print("Budget-Constrained Transfer Recommendations:")
for out_player, in_player in position_transfers:
    print(f"OUT: {out_player['web_name']} ({out_player['team']}, {out_player['position']}, {out_player['now_cost']})")
    print(f"IN: {in_player['web_name']} ({in_player['team']}, {in_player['position']}, {in_player['now_cost']})\n")

print("\nHypothetical Transfers (Ignoring Budget):")
for out_player, in_player in hypothetical_transfers:
    print(f"OUT: {out_player['web_name']} ({out_player['team']}, {out_player['position']}, {out_player['now_cost']})")
    print(f"IN: {in_player['web_name']} ({in_player['team']}, {in_player['position']}, {in_player['now_cost']})\n")


Budget-Constrained Transfer Recommendations:
OUT: Gabriel (Arsenal, DEF, 62)
IN: Tarkowski (Everton, DEF, 48)

OUT: Raya (Arsenal, GKP, 55)
IN: Arrizabalaga (Bournemouth, GKP, 45)

OUT: Watkins (Aston Villa, FWD, 90)
IN: Al-Hamadi (Ipswich, FWD, 48)

OUT: Mbeumo (Brentford, MID, 78)
IN: Lavia (Chelsea, MID, 45)

OUT: B.Fernandes (Man Utd, MID, 85)
IN: Lavia (Chelsea, MID, 45)

OUT: Isak (Newcastle, FWD, 85)
IN: Al-Hamadi (Ipswich, FWD, 48)


Hypothetical Transfers (Ignoring Budget):
OUT: Gabriel (Arsenal, DEF, 62)
IN: Tarkowski (Everton, DEF, 48)

OUT: Raya (Arsenal, GKP, 55)
IN: Arrizabalaga (Bournemouth, GKP, 45)

OUT: Watkins (Aston Villa, FWD, 90)
IN: Al-Hamadi (Ipswich, FWD, 48)

OUT: Mbeumo (Brentford, MID, 78)
IN: Lavia (Chelsea, MID, 45)

OUT: B.Fernandes (Man Utd, MID, 85)
IN: Lavia (Chelsea, MID, 45)

OUT: Isak (Newcastle, FWD, 85)
IN: Al-Hamadi (Ipswich, FWD, 48)



In [ ]:
import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable
import requests

# Load the dataset of all players
players_df = pd.read_csv("players.csv")  # Ensure this file contains all player data

# Manual input of player IDs in your current squad
current_squad_ids = [15, 291, 630, 3, 366, 99, 491, 328, 541, 58, 401, 521, 177, 295, 270]  # Replace with your current squad's player IDs

# Filter the current squad details from the dataset
current_squad_df = players_df[players_df['id'].isin(current_squad_ids)]

# Fetch fixture data from FPL API
url = "https://fantasy.premierleague.com/api/fixtures/"
response = requests.get(url)
fixtures = response.json()

# Convert fixtures to DataFrame
fixtures_df = pd.DataFrame(fixtures)
fixtures_df = fixtures_df[['event', 'team_h', 'team_a', 'team_h_difficulty', 'team_a_difficulty']]

# Team ID to name mapping for the 2024/25 Premier League season
team_mapping = {
    1: "Arsenal", 2: "Aston Villa", 3: "Bournemouth", 4: "Brentford", 5: "Brighton & Hove Albion",
    6: "Chelsea", 7: "Crystal Palace", 8: "Everton", 9: "Fulham", 10: "Liverpool",
    11: "Manchester City", 12: "Manchester United", 13: "Newcastle United", 14: "Nottingham Forest",
    15: "Tottenham Hotspur", 16: "West Ham United", 17: "Wolverhampton Wanderers", 18: "Leicester City",
    19: "Ipswich Town", 20: "Southampton"
}

# Map team IDs to names safely
fixtures_df.loc[:, 'home_team'] = fixtures_df['team_h'].map(team_mapping)
fixtures_df.loc[:, 'away_team'] = fixtures_df['team_a'].map(team_mapping)

# Calculate average fixture difficulty for each team
current_gameweek = 14  # Replace with current Gameweek
n = 5  # Number of upcoming Gameweeks to consider
upcoming_fixtures = fixtures_df[fixtures_df['event'] <= current_gameweek + n]

team_fdr = {}
for team in team_mapping.values():
    home_fixtures = upcoming_fixtures[upcoming_fixtures['home_team'] == team]
    away_fixtures = upcoming_fixtures[upcoming_fixtures['away_team'] == team]
    avg_difficulty = (home_fixtures['team_h_difficulty'].mean() + away_fixtures['team_a_difficulty'].mean()) / 2
    team_fdr[team] = avg_difficulty

# Filter out inactive or irrelevant players
players_df = players_df[players_df['chance_of_playing_this_round'] > 50]
players_df = players_df[players_df['form'] > 0]

# Handle missing columns for expected points calculation
required_columns = ['form', 'points_per_game', 'expected_goals_per_90', 'expected_assists_per_90']
for col in required_columns:
    if col not in players_df.columns:
        players_df[col] = 0  # Add missing columns with default values

# Calculate expected points with FDR adjustment
players_df['expected_points'] = (
    players_df['form'].astype(float) * 0.5 +
    players_df['points_per_game'].astype(float) * 0.3 +
    (players_df.get('expected_goals_per_90', 0).astype(float) + players_df.get('expected_assists_per_90', 0).astype(float)) * 0.2
) * (6 - players_df['team'].map(team_fdr)) / 5  # Adjust for FDR

# Handle NaN or inf values in expected_points
players_df['expected_points'] = players_df['expected_points'].replace([pd.NA, None, float('inf'), -float('inf')], 0).fillna(0)

# Define optimization problem
budget = 100  # Your remaining budget in 0.1M units (e.g., 100 = £100M)
free_transfers = 1  # Number of free transfers available

def optimize_squad(players_df, budget):
    problem = LpProblem("FPL_Squad_Optimization", LpMaximize)

    # Create decision variables for player inclusion
    players = {i: LpVariable(f"player_{i}", cat="Binary") for i in players_df.index}

    # Objective: Maximize expected points
    problem += sum(players_df.loc[i, "expected_points"] * players[i] for i in players_df.index), "Total_Expected_Points"

    # Budget constraint
    problem += sum(players_df.loc[i, "now_cost"] * players[i] for i in players_df.index) <= budget, "Budget_Limit"

    # Squad size constraint
    problem += sum(players[i] for i in players_df.index) == 15, "Squad_Size_Limit"

    # Position constraints
    position_limits = {"DEF": 5, "MID": 5, "FWD": 3, "GKP": 2}
    for position, max_count in position_limits.items():
        problem += sum(players[i] for i in players_df[players_df["position"] == position].index) <= max_count, f"Max_{position}"

    # Solve the optimization problem
    problem.solve()

    # Extract the optimal solution
    selected_players = [i for i in players_df.index if players[i].value() == 1]
    optimal_squad = players_df.loc[selected_players]
    return optimal_squad

# Run optimization
optimized_squad = optimize_squad(players_df, budget)

# Transfer recommendations with position and budget constraints
def get_transfer_recommendations(current_squad_df, optimized_squad, free_transfers, budget):
    current_ids = set(current_squad_df['id'])
    optimized_ids = set(optimized_squad['id'])

    # Players to transfer out and in
    transfers_out = current_ids - optimized_ids
    transfers_in = optimized_ids - current_ids

    out_players = players_df[players_df['id'].isin(transfers_out)]
    in_players = players_df[players_df['id'].isin(transfers_in)]

    # Position-matching transfers
    position_transfers = []
    remaining_budget = budget + out_players['now_cost'].sum()

    for _, out_player in out_players.iterrows():
        position = out_player['position']
        possible_replacements = in_players[in_players['position'] == position]
        affordable_replacements = possible_replacements[possible_replacements['now_cost'] <= remaining_budget]

        if not affordable_replacements.empty:
            best_replacement = affordable_replacements.sort_values(by='expected_points', ascending=False).iloc[0]
            position_transfers.append((out_player, best_replacement))
            remaining_budget -= best_replacement['now_cost']

    # Hypothetical transfers ignoring budget
    hypothetical_transfers = [
        (out_player, in_players[in_players['position'] == out_player['position']].sort_values(by='expected_points', ascending=False).iloc[0])
        for _, out_player in out_players.iterrows()
        if not in_players[in_players['position'] == out_player['position']].empty
    ]

    return position_transfers, hypothetical_transfers

position_transfers, hypothetical_transfers = get_transfer_recommendations(
    current_squad_df, optimized_squad, free_transfers, budget
)

# Display results
print("Budget-Constrained Transfer Recommendations:")
for out_player, in_player in position_transfers:
    print(f"OUT: {out_player['web_name']} ({out_player['team']}, {out_player['position']}, {out_player['now_cost']})")
    print(f"IN: {in_player['web_name']} ({in_player['team']}, {in_player['position']}, {in_player['now_cost']})\n")

print("\nHypothetical Transfers (Ignoring Budget):")
for out_player, in_player in hypothetical_transfers:
    print(f"OUT: {out_player['web_name']} ({out_player['team']}, {out_player['position']}, {out_player['now_cost']})")
    print(f"IN: {in_player['web_name']} ({in_player['team']}, {in_player['position']}, {in_player['now_cost']})\n")


Budget-Constrained Transfer Recommendations:
OUT: Gabriel (Arsenal, DEF, 62)
IN: Tarkowski (Everton, DEF, 48)

OUT: Raya (Arsenal, GKP, 55)
IN: Arrizabalaga (Bournemouth, GKP, 45)

OUT: Watkins (Aston Villa, FWD, 90)
IN: Al-Hamadi (Ipswich, FWD, 48)

OUT: Mbeumo (Brentford, MID, 78)
IN: Lavia (Chelsea, MID, 45)

OUT: B.Fernandes (Man Utd, MID, 85)
IN: Lavia (Chelsea, MID, 45)

OUT: Isak (Newcastle, FWD, 85)
IN: Al-Hamadi (Ipswich, FWD, 48)


Hypothetical Transfers (Ignoring Budget):
OUT: Gabriel (Arsenal, DEF, 62)
IN: Tarkowski (Everton, DEF, 48)

OUT: Raya (Arsenal, GKP, 55)
IN: Arrizabalaga (Bournemouth, GKP, 45)

OUT: Watkins (Aston Villa, FWD, 90)
IN: Al-Hamadi (Ipswich, FWD, 48)

OUT: Mbeumo (Brentford, MID, 78)
IN: Lavia (Chelsea, MID, 45)

OUT: B.Fernandes (Man Utd, MID, 85)
IN: Lavia (Chelsea, MID, 45)

OUT: Isak (Newcastle, FWD, 85)
IN: Al-Hamadi (Ipswich, FWD, 48)

